In [1]:
import pandas as pd
import numpy as np
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO    

In [132]:
def fpage(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr,  laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 1
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,caching=caching, check_extractable=False):
        interpreter.process_page(page)

    text = retstr.getvalue()
    #return text.lstrip().split("\n")

    ks=[c.upper() for c in text.lstrip().split("\n") [:50] if "" !=c ]
    print(ks)
    
    lst=[]
    try:
        for j in ks:
            if "TRANSACTION TYPE" in j:
                index_no = ks.index(j)
                for k in ks[0:index_no+1]:
                    if k not in lst:
                        lst.append(k)
                return lst
            elif "ACCOUNT STATEMENT FROM" in j:
                index_no = ks.index(j)
                for k in ks[0:index_no+1]:
                    if k not in lst:
                        lst.append(k)
                return lst
                
    except:
        pass                 

f = "sbi10.pdf"
cst_info = fpage(f)
cst_info

['(CID:9)    (CID:9)', '(CID:9)   :(CID:9) 80 RADHA GOVIND BHAWAN', '(CID:9)MADAN BADI GYAN DEEP SCHOOL', '(CID:9)NEW KHEDA JAIPUR-302012', '(CID:9)JAIPUR', 'ACCOUNT NAME                   :(CID:9)MR. MUKUL  SHARMA', 'ADDRESS                        (CID:9)', '                               (CID:9)', '                                         (CID:9)', '                                         (CID:9)', 'DATE                                    :(CID:9)22 JAN 2020', 'ACCOUNT NUMBER                 :(CID:9)00000051104691138', 'ACCOUNT DESCRIPTION            :(CID:9)REGULAR SB CHQ-INDIVIDUALS URAL-INR', 'BRANCH                                :(CID:9)JAIPUR KALWAR ROAD', 'DRAWING POWER                   :(CID:9)0.00', 'INTEREST RATE(% P.A.)          :(CID:9)3.25', 'MOD BALANCE                     :(CID:9)0.00', 'CIF NO.                               :(CID:9)71157413727', 'IFS CODE                            :(CID:9)SBIN0031797', 'MICR CODE                        :(CID:9)302002140', 'NOMINATI

['(CID:9)    (CID:9)',
 '(CID:9)   :(CID:9) 80 RADHA GOVIND BHAWAN',
 '(CID:9)MADAN BADI GYAN DEEP SCHOOL',
 '(CID:9)NEW KHEDA JAIPUR-302012',
 '(CID:9)JAIPUR',
 'ACCOUNT NAME                   :(CID:9)MR. MUKUL  SHARMA',
 'ADDRESS                        (CID:9)',
 '                               (CID:9)',
 '                                         (CID:9)',
 'DATE                                    :(CID:9)22 JAN 2020',
 'ACCOUNT NUMBER                 :(CID:9)00000051104691138',
 'ACCOUNT DESCRIPTION            :(CID:9)REGULAR SB CHQ-INDIVIDUALS URAL-INR',
 'BRANCH                                :(CID:9)JAIPUR KALWAR ROAD',
 'DRAWING POWER                   :(CID:9)0.00',
 'INTEREST RATE(% P.A.)          :(CID:9)3.25',
 'MOD BALANCE                     :(CID:9)0.00',
 'CIF NO.                               :(CID:9)71157413727',
 'IFS CODE                            :(CID:9)SBIN0031797',
 'MICR CODE                        :(CID:9)302002140',
 'NOMINATION REGISTERED      :(CID:9)YES',


In [6]:
def tabu(f):
    tables=tabula.read_pdf(f,
                      lattice=True,
                      pages='all',
                      silent=True,
                      multiple_tables=True,
                      pandas_options={'header':None})

    df = pd.DataFrame()
    df = pd.concat([c for c in tables]).drop_duplicates()
    return df

f = "sbi10.pdf"
parse = tabu(f)


# Practice

In [133]:
tables=tabula.read_pdf("icici1.pdf",
                      lattice=True,
                      pages='all',
                      silent=True,
                      multiple_tables=True,
                      pandas_options={'header':None})

df = pd.DataFrame()
df = pd.concat([c for c in tables]).drop_duplicates()
df.head()

,0,1,2,3,4,5,6,7
0,Sr No,Value Date,Transactio\rn Date,Cheque\rNumber,Transactio\rn Remarks,Debit\rAmount,Credit\rAmount,Balance(IN\rR)
1,1,12-Jun-\r2019,12-Jun-\r2019,NaN,CLG/MUK\rUL\rSHARMA/5\r94740/SBI/\r01.06.2019,NA,50000.00,50000.00
2,2,13-Jun-\r2019,13-Jun-\r2019,NaN,NEFT-\rN1641908\r50119453-\rEVOLVE\rTECHNOL\rO...,NA,56078.00,106078.00
3,3,13-Jun-\r2019,13-Jun-\r2019,NaN,MMT/IMPS\r/91641058\r5181/SBIN\r0031797,25000.00,NA,81078.00
4,4,13-Jun-\r2019,13-Jun-\r2019,NaN,MMT/IMPS\r/91641058\r5181/SBIN\r0031797,NA,25000.00,106078.00


In [134]:
try:
    idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
    df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
except:
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('description').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        print("\nSBI-Column Headers Missing"); pass

In [135]:
df.head()

,Sr No,Value Date,Transactio\rn Date,Cheque\rNumber,Transactio\rn Remarks,Debit\rAmount,Credit\rAmount,Balance(IN\rR)
0,1,12-Jun-\r2019,12-Jun-\r2019,NaN,CLG/MUK\rUL\rSHARMA/5\r94740/SBI/\r01.06.2019,NA,50000.00,50000.00
1,2,13-Jun-\r2019,13-Jun-\r2019,NaN,NEFT-\rN1641908\r50119453-\rEVOLVE\rTECHNOL\rO...,NA,56078.00,106078.00
2,3,13-Jun-\r2019,13-Jun-\r2019,NaN,MMT/IMPS\r/91641058\r5181/SBIN\r0031797,25000.00,NA,81078.00
3,4,13-Jun-\r2019,13-Jun-\r2019,NaN,MMT/IMPS\r/91641058\r5181/SBIN\r0031797,NA,25000.00,106078.00
4,5,13-Jun-\r2019,13-Jun-\r2019,NaN,MMT/IMPS\r/91641178\r4486/SBIN\r0031797,25000.00,NA,81078.00


In [136]:
try:
    df = df.drop(["Init."], axis=1)
except:
    try:
        df = df.drop(["Branch Name"], axis=1)
    except:pass
    
try:
    df = df[~df.index.isin(df[df.apply(lambda row:row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance').any(),axis=1)==True].index)]
    #df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False)==1)].index,axis=1,inplace=True)
except:
    pass

In [137]:
df.head()

,Sr No,Value Date,Transactio\rn Date,Cheque\rNumber,Transactio\rn Remarks,Debit\rAmount,Credit\rAmount,Balance(IN\rR)
0,1,12-Jun-\r2019,12-Jun-\r2019,NaN,CLG/MUK\rUL\rSHARMA/5\r94740/SBI/\r01.06.2019,NA,50000.00,50000.00
1,2,13-Jun-\r2019,13-Jun-\r2019,NaN,NEFT-\rN1641908\r50119453-\rEVOLVE\rTECHNOL\rO...,NA,56078.00,106078.00
2,3,13-Jun-\r2019,13-Jun-\r2019,NaN,MMT/IMPS\r/91641058\r5181/SBIN\r0031797,25000.00,NA,81078.00
3,4,13-Jun-\r2019,13-Jun-\r2019,NaN,MMT/IMPS\r/91641058\r5181/SBIN\r0031797,NA,25000.00,106078.00
4,5,13-Jun-\r2019,13-Jun-\r2019,NaN,MMT/IMPS\r/91641178\r4486/SBIN\r0031797,25000.00,NA,81078.00


In [139]:
df.to_csv("parsed.csv",index=0)